In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
ORG_DIR = '/kaggle/input/indoor-location-navigation'
sub = pd.read_csv(ORG_DIR+"/sample_submission.csv")
sub

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0


In [3]:
DATA_DIR = '/kaggle/input/indoor-competition-dataset'
os.listdir(DATA_DIR)

['5da138364db8ce0c98bc00f1_train.csv',
 '5d27097f03f801723c320d97_train.csv',
 '5da958dd46f8266d0737457b_test.csv',
 '5d2709b303f801723c327472_test.csv',
 '5da138754db8ce0c98bca82f_test.csv',
 '5d27075f03f801723c2e360f_test.csv',
 '5dbc1d84c1eb61796cf7c010_test.csv',
 '5c3c44b80379370013e0fd2b_train.csv',
 '5d27075f03f801723c2e360f_train.csv',
 '5d2709e003f801723c32d896_test.csv',
 '5da1382d4db8ce0c98bbe92e_train.csv',
 '5d2709b303f801723c327472_train.csv',
 '5da138b74db8ce0c98bd4774_train.csv',
 '5d2709e003f801723c32d896_train.csv',
 '5dc8cea7659e181adb076a3f_train.csv',
 '5da138314db8ce0c98bbf3a0_test.csv',
 '5da138764db8ce0c98bcaa46_train.csv',
 '5da138364db8ce0c98bc00f1_test.csv',
 '5da1383b4db8ce0c98bc11ab_test.csv',
 '5dc8cea7659e181adb076a3f_test.csv',
 '5da138b74db8ce0c98bd4774_test.csv',
 '5d2709c303f801723c3299ee_train.csv',
 '5d27099f03f801723c32511d_test.csv',
 '5d2709d403f801723c32bd39_train.csv',
 '5da1383b4db8ce0c98bc11ab_train.csv',
 '5da138274db8ce0c98bbd3d2_train.csv'

In [4]:
data = pd.read_csv(DATA_DIR+'/5da138364db8ce0c98bc00f1_train.csv')
data.columns

Index(['079c8efe6f440818de14103c5bb196d7217602e8',
       '07b0bbdb7132359b74874443328c5b6005c6ac40',
       '0b0b8fd432c70e6e82f4d2ff8e3a3d67b6d8b9d0',
       '0befcde4a78a6aec38e9a25af2c4ce7cffa569d4',
       '0c4f9389bc079285d34b44d9efa90c1d7504fa3b',
       '0ebb75098d09efbec391c9e57481c207a3ece6be',
       '14d93d45757ce00afade8be40f251050820f6586',
       '176a01a196068d07717669cdde238ecda577bdbf',
       '1b467e6893c2c8b5833e54ded5f1db7808db25ed',
       '2505e1d07c36eff2f38531f04c6a69234372a414',
       '267df051cc8183ab5cf930f6f015fb65be336722',
       '2a8dc608d8243b5696d29cf931121d1aad4eb0d1',
       '31ddd0d8108ca66692863632a19edbb52a0b0d49',
       '354d3ccd31b3427ee1414a3365f900666a0e11bc',
       '3a07833d5b75b2cf26e25677b45455766a98143b',
       '3a6228a74967f773668a1cfed31d8ddd30abe71f',
       '3bf03863805ea1bfc417387d3081366c9c0f841a',
       '56bac8f84f491b0f687b6c90605d46d121934f4e',
       '5759c15b51df9f770cc2d5132bb4958006ea29b3',
       '57ca4c27a6ff76a7fae8e63